In [25]:
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import *
import torch as T
import torch.nn as nn
import random
import os
project_path= os.path.dirname(os.path.abspath(os.curdir))
import sys
sys.path.insert(0, project_path+ '/Tools')
sys.path.insert(1, project_path+ '/Optimal Control Methods/Learning Methods/Model Free RL')
sys.path.insert(2, project_path+ '/Systems')
from EnforceTyping import enforce_method_typing
from ParticlesandFields import Field, ClassicalParticle, ParticleInField
from DDPG import DDPGAgent, DDPGAlgorithm
T.Tensor.ndim = property(lambda self: len(self.shape))

In [26]:
coulomb_constant = 8.9875e9  # N*m^2/C^2
@dataclass(kw_only=True)
class ElectrostaticField2D(Field):
  """
  A class used to represent a 2D Electrostatic Field

  Attributes
  ----------
  field_sources: dict
      a formatted string to print out what the animal says
  dimensionality: tuple
      a tuple of the dimensionality of the field  

  Methods
  -------
  dynamics(self, observation_position: np.ndarray, time: float) -> np.ndarray:
      Represents the value of the field at any given point(s) or time. 
  potential(self, observation_position: np.ndarray, time: float) -> float:
      Represents the potential due to the field at a given position and/or time  
  potential_difference(self, initial_position: np.ndarray, final_position: np.ndarray, time: float) -> float:
      Represents the potential difference between two positions at a given time in the vector field   
  gradient(self, observation_position: np.ndarray, time: float) -> float:
      Represents the gradient at a given position and/or time in the vector field 
  curl(self, observation_position: np.ndarray, time: float) -> float:
      Represents the curl at a given position and/or time in the vector field 
  divergence(self, observation_position: np.ndarray, time: float) -> float:
      Represents the divergence at a given position and/or time in the vector field
  """
  field_sources: dict
  dimensionality: tuple = (2,)

  def __call__(self, observation_position: np.ndarray) -> np.ndarray:
      return self.dynamics(observation_position)

  def __post_init__(self):
    assert len(self.field_sources["Particle"]) == len(self.field_sources["Position"]), "The length of particles and fields don't match"
    for field_source, _ in zip(self.field_sources["Particle"], self.field_sources["Position"]):
        assert isinstance(field_source, ClassicalParticle),  "The field source is not a particle" 

  @enforce_method_typing
  def dynamics(self, observation_position: np.ndarray) -> np.ndarray:
      """
      This function outputs the field strength due to field sources experienced at any given point(s) or time. 
      This determines the physics of the field (a 2D Electricstatic Field in this case)

      Args:
          observation_position (np.ndarray): The position.

      Returns:
          np.ndarray: The electric field strength vector at the given position.
      """
      electric_field_vector = np.zeros_like(observation_position)
      for field_source, source_position in zip(self.field_sources["Particle"], self.field_sources["Position"]):
          position_vectors = np.broadcast_to(source_position, reversed(observation_position.shape)).T
          displacement_vectors = observation_position - position_vectors
          displacement_magnitude = np.linalg.norm(displacement_vectors, axis=0)
          electric_field_vector += (displacement_vectors * field_source.charge) / displacement_magnitude**3
      electric_field_vector = coulomb_constant * electric_field_vector
      return np.round(electric_field_vector, 3)  # N/C or V/m

  @enforce_method_typing
  def potential(self, observation_position: np.ndarray) -> float:
      """
      Calculate the potential (voltage) at a position in the field.

      Args:
          observation_position (np.ndarray): The position.

      Returns:
          np.ndarray: The electric potential at the given position.
      """
      electric_potential = 0.0
      for field_source, source_position in zip(self.field_sources["Particle"], self.field_sources["Position"]):
          position_vectors = np.broadcast_to(source_position, reversed(observation_position.shape)).T
          displacement_vectors = observation_position - position_vectors
          displacement_magnitude = np.linalg.norm(displacement_vectors, axis=0)
          electric_potential += field_source.charge / displacement_magnitude
      electric_potential = coulomb_constant * electric_potential
      return np.round(electric_potential, 3)  # V

  @enforce_method_typing
  def potential_difference(self, initial_position: np.ndarray, final_position: np.ndarray) -> float:
    """
    Calculate the potential difference between the initial position and the final position in the field.

    Args:
        initial_position (np.ndarray): The starting position.
        final_position (np.ndarray): The ending position.
        resolution (int, optional): The number of intervals to divide the path into. Defaults to 5000.

    Returns:
        float: The work required to move from the initial position to the final position.
    """
    assert initial_position.shape == self.dimensionality, "initial_position has the wrong dimensions"
    assert final_position.shape == self.dimensionality, "final_position has the wrong dimensions"
    PorentialDifference= self.potential(initial_position)- self.potential(final_position)
    return PorentialDifference

  @enforce_method_typing
  def gradient(self, observation_position: np.ndarray, delta: float= 0.001)->np.ndarray:
    """
    This function returns the derivative of the field at a given point

    Args:
        observation_position (np.ndarray): The position.
        delta (float, optional): The step size. Defaults to 0.001.

    Returns: 
      np.ndarray: The gradient of the field at the given position.
    """
    gradient= np.zeros_like(observation_position)
    for i in range(len(observation_position)):
      di= np.zeros_like(observation_position)
      di[i, ] = di[i, ]+delta
      plusdi= observation_position+ di
      minusdi= observation_position- di
      gradient[i]= (self.dynamics(plusdi)- self.dynamics(minusdi))[i]/ (2* delta)
    return gradient

  @enforce_method_typing
  def plot_field(self, low_bound= -5, high_bound= 5, n_vectors= 50):
    """
    This function plots the 2D electric vector field

    Args:
    low_bound (float, optional): The lower bound of the plot. Defaults to -5.
    high_bound (float, optional): The upper bound of the plot. Defaults to 5.
    n_vectors (int, optional): The number of vectors to plot. Defaults to 50.

    """
    observation_position= np.meshgrid(np.linspace(low_bound, high_bound, n_vectors), 
                                    np.linspace(low_bound, high_bound, n_vectors))
    observation_position= np.stack(observation_position)
    xd, yd = self.dynamics(observation_position)
    xd = xd / np.sqrt(xd**2 + yd**2)
    yd = yd / np.sqrt(xd**2 + yd**2)
    color_aara = np.sqrt(xd**2+ yd**2)
    fig, ax = plt.subplots(1,1)
    cp = ax.quiver(observation_position[0],observation_position[1],xd,yd,color_aara)
    fig.colorbar(cp)
    plt.rcParams['figure.dpi'] = 150
    plt.show()

In [27]:
negative_charge= ClassicalParticle(mass=1.0, charge= -1e-9)
positive_charge= ClassicalParticle(mass=1.0, charge= 1e-9)
sources = {"Particle": [negative_charge],
           "Position": [np.array([0.0, 0.0])]} 
target= np.array([5.0, 0.0])
test_electric_field= ElectrostaticField2D(field_sources=sources)
point_charge_in_electric_field= ParticleInField(field=test_electric_field, 
                                               particle=negative_charge, 
                                               target=target)

In [28]:
test_actor_layer= (10, 5)
test_critic_layer= (10, 5)
test_actor_activations= (nn.ReLU(), nn.ReLU())
test_critic_activations= (nn.ReLU(), nn.ReLU())
test_agent = DDPGAgent(point_charge_in_electric_field,
                       actor_layers=test_actor_layer, 
                       critic_layers=test_critic_layer,
                       actor_activations= test_actor_activations,
                       critic_activations= test_critic_activations,
                       observation_size= 4,
                       action_size= 2,
                       actor_learning_rate= 0.01,
                       critic_learning_rate= 0.01,
                       soft_update_rate=0.01,
                       control_interval=0.5,
                       control_magnitude= 1.0,
                       batch_size=10,
                       max_size=500)

In [29]:
test_state= test_agent.observe() 

test_action= test_agent.act(test_state)
test_value= test_agent.critic.forward(test_state, test_action)
# print(test_agent.policy)
print(test_state)
print(test_action)
print(test_value)

tensor([-5.5578, -2.7034,  0.0000,  0.0000])
tensor([-0.7633,  0.6759])
tensor([-0.7679], grad_fn=<ViewBackward0>)


In [30]:
print(test_agent.memory)
curr_state= ParticleInField.State(position=np.array([-5.0, 0.0]), 
                                  velocity=np.array([0.0, 0.0]))
for _ in range(50):
    rand_observation= test_agent.observe(curr_state)
    rand_action= test_agent.act(rand_observation)
    next_state, rand_reward, rand_terminal_signal= point_charge_in_electric_field.transition_step(curr_state, np.array(rand_action), test_agent.control_interval) 
    test_agent.memory.append((rand_observation, 
                              rand_action, 
                              test_agent.observe(next_state), 
                              rand_reward, 
                              rand_terminal_signal))
    curr_state= next_state
print(test_agent.memory)
(test_electric_field.dynamics(np.array([5.0, 0.0]))* positive_charge.charge)

deque([], maxlen=500)
deque([(tensor([-5.,  0.,  0.,  0.]), tensor([-0.6589,  0.6743]), tensor([-5.0824,  0.0843, -0.3294,  0.3372]), -0.5271413242082232, False), (tensor([-5.0824,  0.0843, -0.3294,  0.3372]), tensor([-0.6638,  0.6803]), tensor([-5.3301,  0.3379, -0.6613,  0.6773]), -0.7045999440541877, False), (tensor([-5.3301,  0.3379, -0.6613,  0.6773]), tensor([-0.6557,  0.6709]), tensor([-5.7427,  0.7604, -0.9892,  1.0128]), -0.874046152537451, False), (tensor([-5.7427,  0.7604, -0.9892,  1.0128]), tensor([-0.6339,  0.6508]), tensor([-6.3165,  1.3482, -1.3062,  1.3382]), -1.0396523550321266, False), (tensor([-6.3165,  1.3482, -1.3062,  1.3382]), tensor([-0.6288,  0.6375]), tensor([-7.0482,  2.0970, -1.6206,  1.6569]), -1.233678629546679, False), (tensor([-7.0482,  2.0970, -1.6206,  1.6569]), tensor([-0.6314,  0.6102]), tensor([-7.9374,  3.0017, -1.9363,  1.9620]), -1.4372817391246215, False), (tensor([-7.9374,  3.0017, -1.9363,  1.9620]), tensor([-0.5948,  0.6313]), tensor([-8.979

array([-3.6e-10,  0.0e+00])

In [67]:
batch = random.sample(test_agent.memory, test_agent.batch_size)
observations, actions, next_observations, rewards, dones = zip(*batch)

state = T.stack(observations).to(test_agent.critic.device)
print('state:', state)

action = T.stack(actions).to(test_agent.critic.device)
print('action:', action)

reward = T.tensor(rewards, dtype=T.float).unsqueeze(1).to(test_agent.critic.device)
print('reward:', reward)

new_state = T.stack(next_observations).to(test_agent.critic.device)
print('new_state:', new_state)

done = T.tensor(dones, dtype=T.float).unsqueeze(1).to(test_agent.critic.device)
print('done:', done)

target_actions = test_agent.actor.forward(new_state)
print('target_actions:', target_actions)

critic_value_ = test_agent.target_critic.forward(new_state, target_actions) 
print('critic_value:', critic_value_)

q_expected = test_agent.critic.forward(state, action)
print('q_expected:', q_expected)

q_targets = reward + test_agent.discount_rate * critic_value_ * (1 - done)
print('q_targets:', q_targets)

state: tensor([[ 1.0000e+00,  6.0790e-19,  2.2675e+08,  2.0820e+03],
        [ 1.0000e+00, -5.0025e-18,  8.6675e+07,  9.0796e+02],
        [ 1.0000e+00, -1.2564e-18,  1.5545e+08,  1.9562e+03],
        [ 1.0000e+00, -4.0089e-18,  1.0147e+08,  1.1841e+03],
        [ 1.0000e+00, -9.2481e-18,  4.8376e+07,  2.9124e+02],
        [ 1.0000e+00, -7.4775e-19,  1.7077e+08,  2.0811e+03],
        [ 1.0000e+00, -1.4741e-19,  1.9296e+08,  2.1712e+03],
        [ 1.0000e+00, -4.1709e-18,  9.8954e+07,  1.1381e+03],
        [ 1.0000e+00, -4.8319e-18,  8.9111e+07,  9.5403e+02],
        [ 1.0000e+00, -4.3806e-19,  1.8172e+08,  2.1397e+03]])
action: tensor([[ 5.9034e-08, -5.3228e-08],
        [ 6.0053e-08, -5.3353e-08],
        [ 6.2231e-08, -5.3162e-08],
        [ 6.1101e-08, -4.5831e-08],
        [ 6.6462e-08, -5.4459e-08],
        [ 5.8999e-08, -5.5274e-08],
        [ 5.6896e-08, -5.4522e-08],
        [ 6.1845e-08, -4.5301e-08],
        [ 5.8243e-08, -5.1368e-08],
        [ 5.8730e-08, -5.3340e-08]])
rew

In [ ]:
critic_loss = nn.MSELoss()(q_expected, q_targets.detach())
test_agent.critic.train()
test_agent.critic.optimizer.zero_grad()
critic_loss.backward()
test_agent.critic.optimizer.step()

mu = test_agent.actor.forward(state)
Actor_loss = -test_agent.critic.forward(state, mu)

Actor_loss = T.mean(Actor_loss)
self.actor.train()
self.actor.optimizer.zero_grad()
Actor_loss.backward()
self.actor.optimizer.step()

self.update_network_parameters()

In [ ]:
test_trajectory, test_trajectory_return= test_agent.sample_trajectory(10.0)
test_agent.plot_trajectory(test_trajectory)

In [ ]:
DDPGAlgorithm(point_charge_in_electric_field, test_agent, 800, 15)

In [ ]:
path, path_return= test_agent.sample_trajectory(10.0)
test_agent.plot_trajectory(path)